In [2]:
import geopandas as gpd

In [11]:
# for file in ['actual', 'future']:
#     filename = f'/app/assets/buildings/{file}.parquet'
#     gdf = gpd.read_parquet(filename)
#     building_cols = ['Pisos', 'Metros', 'plate_id', 'geometry']
#     gdf[building_cols].to_parquet(filename)

In [23]:
filename = '/app/assets/buildings/actual.parquet'
gdf_actual = gpd.read_parquet(filename)
gdf_actual.to_file('/app/data/etiquetado/buildings_actual')
filename = '/app/assets/buildings/future.parquet'
gdf_future = gpd.read_parquet(filename)
gdf_future.to_file('/app/data/etiquetado/buildings_future')

In [14]:
gdf_actual.shape

(5975, 4)

In [17]:
gdf_future.shape

(5035, 4)

In [20]:
diff = gdf_future[gdf_future['geometry'].isin(gdf_actual['geometry'])]

In [22]:
diff['plate_id'].value_counts()

0.0    1227
2.0      38
1.0      33
Name: plate_id, dtype: int64

In [3]:
gdf_actual = gpd.read_file('/app/data/etiquetados_por_plates/etiquetados/buildings/actual')
gdf_future = gpd.read_file('/app/data/etiquetados_por_plates/etiquetados/buildings/future')

In [11]:
building_cols = ['Pisos', 'Metros', 'plate_id', 'geometry']
gdf_actual = gdf_actual[building_cols]
gdf_future = gdf_future[building_cols]

gdf_actual['plate_id'].fillna(0, inplace=True)
gdf_future['plate_id'].fillna(0, inplace=True)

In [20]:
plate_id = 7
gdf = gdf_actual.copy()
gdf = gdf_future.copy()
# mask = gdf['plate_id']==plate_id
# gdf[mask]
gdf

,Pisos,Metros,plate_id,geometry
0,5.0,11.5,5.0,"POLYGON Z ((672815.297 5921766.440 0.000, 6728..."
1,5.0,11.5,5.0,"POLYGON Z ((672834.982 5921728.498 0.000, 6728..."
2,5.0,11.5,5.0,"POLYGON Z ((672884.882 5921619.556 0.000, 6729..."
3,5.0,11.5,5.0,"POLYGON Z ((672912.512 5921631.671 0.000, 6729..."
4,5.0,11.5,5.0,"POLYGON Z ((672942.357 5921649.451 0.000, 6729..."
...,...,...,...,...
5030,1.0,2.3,1.0,"POLYGON Z ((672620.268 5922838.529 0.000, 6726..."
5031,1.0,2.3,0.0,"POLYGON Z ((672603.826 5922858.433 0.000, 6726..."
5032,5.0,11.5,0.0,"POLYGON Z ((672548.796 5922976.512 0.000, 6725..."
5033,2.0,4.6,0.0,"POLYGON Z ((672594.569 5922890.097 0.000, 6725..."


In [25]:
from modules.Buildings import Buildings
import pandas as pd
builds = Buildings()

In [26]:
def _update_plate_area(builds, plate_id, scenario_id):
    
    current_data = builds.current_scenario
    current_data = current_data[current_data['plate_id']!=plate_id]

    new_data = builds.scenarios[scenario_id]
    new_data = new_data[new_data['plate_id']==plate_id]

    update_data = pd.concat([current_data, new_data])
    update_data = gpd.GeoDataFrame(data=update_data.drop(columns=['geometry']), geometry=update_data['geometry'])

    builds.current_scenario = update_data

    return builds

In [28]:
builds.current_scenario.shape

(5975, 4)

In [29]:
builds = _update_plate_area(builds, 7, 1)

In [30]:
builds.current_scenario.shape

(5739, 4)